In [1]:
import pandas as pd
import networkx as nx
from sodapy import Socrata
import math
import random
import matplotlib.pyplot as plt
import numpy as np
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import copy 

#to supress some annoying warnings
# import warnings; warnings.simplefilter('ignore')

In [2]:
# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.cityofnewyork.us", None)

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("2yzn-sicd", limit=20000)

# Convert to pandas DataFrame
data = pd.DataFrame.from_records(results)

In [3]:
data

,dropoff_datetime,dropoff_latitude,dropoff_longitude,extra,fare_amount,mta_tax,passenger_count,payment_type,pickup_datetime,pickup_latitude,pickup_longitude,rate_code,store_and_fwd_flag,tip_amount,tolls_amount,total_amount,trip_distance,vendor_id
0,2015-05-02T03:29:59.000,40.70794677734375,-74.007194519042969,0.5,10,0.5,1,1,2015-05-02T03:19:52.000,40.740711212158203,-74.004623413085938,1,N,2.26,0,13.56,2.58,2
1,2015-01-12T08:20:30.000,0,0,0,8.5,0.5,1,2,2015-01-12T08:11:21.000,0,0,1,N,0,0,9.3,1.67,2
2,2015-04-08T20:19:26.000,40.773033142089844,-73.958457946777344,1,17.5,0.5,1,2,2015-04-08T19:58:20.000,40.727493286132812,-74.003173828125,1,N,0,0,19.3,4.45,2
3,2015-06-10T17:34:50.000,40.756557464599609,-73.958999633789063,1,8,0.5,1,1,2015-06-10T17:26:20.000,40.778423309326172,-73.946426391601563,1,N,2.45,0,12.25,1.57,2
4,2015-04-11T07:26:43.000,40.716964721679688,-74.052459716796875,0,0,0,2,3,2015-04-11T07:09:13.000,40.723400115966797,-74.007278442382813,5,N,0,0,0.3,7.4,1
5,2015-01-19T11:37:08.000,40.788333892822266,-73.976715087890625,0,9,0.5,1,2,2015-01-19T11:25:37.000,40.768276214599609,-73.981735229492188,1,N,0,0,9.8,1.6,1
6,2015-06-30T06:30:21.000,40.647281646728516,-73.788742065429687,0,52,0.5,5,1,2015-06-30T06:04:19.000,40.770626068115234,-73.953964233398438,2,N,12,5.54,70.34,18.88,2
7,2015-02-25T13:43:30.000,40.783622741699219,-73.950439453125,0,3.5,0.5,6,2,2015-02-25T13:42:23.000,40.778621673583984,-73.954345703125,1,N,0,0,4.3,0.41,2
8,2015-01-30T21:16:34.000,40.773704528808594,-73.981414794921875,0.5,5,0.5,1,1,2015-01-30T21:12:14.000,40.781723022460938,-73.98089599609375,1,N,1,0,7.3,0.65,2
9,2015-05-21T12:18:12.000,40.773185729980469,-73.885536193847656,0,35.5,0.5,1,1,2015-05-21T11:52:32.000,40.70587158203125,-74.005844116210937,1,N,0,5.54,41.84,12.14,2


In [4]:
geolocator = Nominatim(user_agent="my-application")
reverse = RateLimiter(geolocator.reverse, min_delay_seconds=2)

In [5]:
def to_zip(lat,lon):
    cd = copy.deepcopy(lat)
    cd += ","
    cd += copy.deepcopy(lon)
#     print("query: ", cd)
    loc = reverse(cd)
    try:
        res = int(loc.raw['address']['postcode'])
        return res 
    except:
        return 0

In [6]:
dropoff_zip = []
pickup_zip = []
for i in range(data.shape[0]):
    d_lat = data['dropoff_latitude'][i]
    d_lon = data['dropoff_longitude'][i]
    
    p_lat = data['pickup_latitude'][i]
    p_lon = data['pickup_longitude'][i]
    
    dropoff_zip.append(to_zip(d_lat, d_lon))
    pickup_zip.append(to_zip(p_lat, p_lon))
    
data.insert(loc=len(data.columns),column='dropoff_zip', value=dropoff_zip)
data.insert(loc=len(data.columns),column='pickup_zip', value=pickup_zip)
data

Traceback (most recent call last):
  File "/Users/Bisrat/anaconda/lib/python3.6/urllib/request.py", line 1318, in do_open
    encode_chunked=req.has_header('Transfer-encoding'))
  File "/Users/Bisrat/anaconda/lib/python3.6/http/client.py", line 1239, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "/Users/Bisrat/anaconda/lib/python3.6/http/client.py", line 1285, in _send_request
    self.endheaders(body, encode_chunked=encode_chunked)
  File "/Users/Bisrat/anaconda/lib/python3.6/http/client.py", line 1234, in endheaders
    self._send_output(message_body, encode_chunked=encode_chunked)
  File "/Users/Bisrat/anaconda/lib/python3.6/http/client.py", line 1026, in _send_output
    self.send(msg)
  File "/Users/Bisrat/anaconda/lib/python3.6/http/client.py", line 964, in send
    self.connect()
  File "/Users/Bisrat/anaconda/lib/python3.6/http/client.py", line 1400, in connect
    server_hostname=server_hostname)
  File "/Users/Bisrat/anaconda/lib/python3

Traceback (most recent call last):
  File "/Users/Bisrat/anaconda/lib/python3.6/urllib/request.py", line 1318, in do_open
    encode_chunked=req.has_header('Transfer-encoding'))
  File "/Users/Bisrat/anaconda/lib/python3.6/http/client.py", line 1239, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "/Users/Bisrat/anaconda/lib/python3.6/http/client.py", line 1285, in _send_request
    self.endheaders(body, encode_chunked=encode_chunked)
  File "/Users/Bisrat/anaconda/lib/python3.6/http/client.py", line 1234, in endheaders
    self._send_output(message_body, encode_chunked=encode_chunked)
  File "/Users/Bisrat/anaconda/lib/python3.6/http/client.py", line 1026, in _send_output
    self.send(msg)
  File "/Users/Bisrat/anaconda/lib/python3.6/http/client.py", line 964, in send
    self.connect()
  File "/Users/Bisrat/anaconda/lib/python3.6/http/client.py", line 1392, in connect
    super().connect()
  File "/Users/Bisrat/anaconda/lib/python3.6/http/client.

Traceback (most recent call last):
  File "/Users/Bisrat/anaconda/lib/python3.6/urllib/request.py", line 1318, in do_open
    encode_chunked=req.has_header('Transfer-encoding'))
  File "/Users/Bisrat/anaconda/lib/python3.6/http/client.py", line 1239, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "/Users/Bisrat/anaconda/lib/python3.6/http/client.py", line 1285, in _send_request
    self.endheaders(body, encode_chunked=encode_chunked)
  File "/Users/Bisrat/anaconda/lib/python3.6/http/client.py", line 1234, in endheaders
    self._send_output(message_body, encode_chunked=encode_chunked)
  File "/Users/Bisrat/anaconda/lib/python3.6/http/client.py", line 1026, in _send_output
    self.send(msg)
  File "/Users/Bisrat/anaconda/lib/python3.6/http/client.py", line 964, in send
    self.connect()
  File "/Users/Bisrat/anaconda/lib/python3.6/http/client.py", line 1400, in connect
    server_hostname=server_hostname)
  File "/Users/Bisrat/anaconda/lib/python3

Traceback (most recent call last):
  File "/Users/Bisrat/anaconda/lib/python3.6/urllib/request.py", line 1318, in do_open
    encode_chunked=req.has_header('Transfer-encoding'))
  File "/Users/Bisrat/anaconda/lib/python3.6/http/client.py", line 1239, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "/Users/Bisrat/anaconda/lib/python3.6/http/client.py", line 1285, in _send_request
    self.endheaders(body, encode_chunked=encode_chunked)
  File "/Users/Bisrat/anaconda/lib/python3.6/http/client.py", line 1234, in endheaders
    self._send_output(message_body, encode_chunked=encode_chunked)
  File "/Users/Bisrat/anaconda/lib/python3.6/http/client.py", line 1026, in _send_output
    self.send(msg)
  File "/Users/Bisrat/anaconda/lib/python3.6/http/client.py", line 964, in send
    self.connect()
  File "/Users/Bisrat/anaconda/lib/python3.6/http/client.py", line 1400, in connect
    server_hostname=server_hostname)
  File "/Users/Bisrat/anaconda/lib/python3

Traceback (most recent call last):
  File "/Users/Bisrat/anaconda/lib/python3.6/urllib/request.py", line 1318, in do_open
    encode_chunked=req.has_header('Transfer-encoding'))
  File "/Users/Bisrat/anaconda/lib/python3.6/http/client.py", line 1239, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "/Users/Bisrat/anaconda/lib/python3.6/http/client.py", line 1285, in _send_request
    self.endheaders(body, encode_chunked=encode_chunked)
  File "/Users/Bisrat/anaconda/lib/python3.6/http/client.py", line 1234, in endheaders
    self._send_output(message_body, encode_chunked=encode_chunked)
  File "/Users/Bisrat/anaconda/lib/python3.6/http/client.py", line 1026, in _send_output
    self.send(msg)
  File "/Users/Bisrat/anaconda/lib/python3.6/http/client.py", line 964, in send
    self.connect()
  File "/Users/Bisrat/anaconda/lib/python3.6/http/client.py", line 1400, in connect
    server_hostname=server_hostname)
  File "/Users/Bisrat/anaconda/lib/python3

...

Traceback (most recent call last):
  File "/Users/Bisrat/anaconda/lib/python3.6/urllib/request.py", line 1318, in do_open
    encode_chunked=req.has_header('Transfer-encoding'))
  File "/Users/Bisrat/anaconda/lib/python3.6/http/client.py", line 1239, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "/Users/Bisrat/anaconda/lib/python3.6/http/client.py", line 1285, in _send_request
    self.endheaders(body, encode_chunked=encode_chunked)
  File "/Users/Bisrat/anaconda/lib/python3.6/http/client.py", line 1234, in endheaders
    self._send_output(message_body, encode_chunked=encode_chunked)
  File "/Users/Bisrat/anaconda/lib/python3.6/http/client.py", line 1026, in _send_output
    self.send(msg)
  File "/Users/Bisrat/anaconda/lib/python3.6/http/client.py", line 964, in send
    self.connect()
  File "/Users/Bisrat/anaconda/lib/python3.6/http/client.py", line 1392, in connect
    super().connect()
  File "/Users/Bisrat/anaconda/lib/python3.6/http/client.

Traceback (most recent call last):
  File "/Users/Bisrat/anaconda/lib/python3.6/urllib/request.py", line 1318, in do_open
    encode_chunked=req.has_header('Transfer-encoding'))
  File "/Users/Bisrat/anaconda/lib/python3.6/http/client.py", line 1239, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "/Users/Bisrat/anaconda/lib/python3.6/http/client.py", line 1285, in _send_request
    self.endheaders(body, encode_chunked=encode_chunked)
  File "/Users/Bisrat/anaconda/lib/python3.6/http/client.py", line 1234, in endheaders
    self._send_output(message_body, encode_chunked=encode_chunked)
  File "/Users/Bisrat/anaconda/lib/python3.6/http/client.py", line 1026, in _send_output
    self.send(msg)
  File "/Users/Bisrat/anaconda/lib/python3.6/http/client.py", line 964, in send
    self.connect()
  File "/Users/Bisrat/anaconda/lib/python3.6/http/client.py", line 1392, in connect
    super().connect()
  File "/Users/Bisrat/anaconda/lib/python3.6/http/client.

Traceback (most recent call last):
  File "/Users/Bisrat/anaconda/lib/python3.6/urllib/request.py", line 1318, in do_open
    encode_chunked=req.has_header('Transfer-encoding'))
  File "/Users/Bisrat/anaconda/lib/python3.6/http/client.py", line 1239, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "/Users/Bisrat/anaconda/lib/python3.6/http/client.py", line 1285, in _send_request
    self.endheaders(body, encode_chunked=encode_chunked)
  File "/Users/Bisrat/anaconda/lib/python3.6/http/client.py", line 1234, in endheaders
    self._send_output(message_body, encode_chunked=encode_chunked)
  File "/Users/Bisrat/anaconda/lib/python3.6/http/client.py", line 1026, in _send_output
    self.send(msg)
  File "/Users/Bisrat/anaconda/lib/python3.6/http/client.py", line 964, in send
    self.connect()
  File "/Users/Bisrat/anaconda/lib/python3.6/http/client.py", line 1392, in connect
    super().connect()
  File "/Users/Bisrat/anaconda/lib/python3.6/http/client.

Traceback (most recent call last):
  File "/Users/Bisrat/anaconda/lib/python3.6/urllib/request.py", line 1318, in do_open
    encode_chunked=req.has_header('Transfer-encoding'))
  File "/Users/Bisrat/anaconda/lib/python3.6/http/client.py", line 1239, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "/Users/Bisrat/anaconda/lib/python3.6/http/client.py", line 1285, in _send_request
    self.endheaders(body, encode_chunked=encode_chunked)
  File "/Users/Bisrat/anaconda/lib/python3.6/http/client.py", line 1234, in endheaders
    self._send_output(message_body, encode_chunked=encode_chunked)
  File "/Users/Bisrat/anaconda/lib/python3.6/http/client.py", line 1026, in _send_output
    self.send(msg)
  File "/Users/Bisrat/anaconda/lib/python3.6/http/client.py", line 964, in send
    self.connect()
  File "/Users/Bisrat/anaconda/lib/python3.6/http/client.py", line 1392, in connect
    super().connect()
  File "/Users/Bisrat/anaconda/lib/python3.6/http/client.

KeyboardInterrupt: 

In [ ]:
data.to_pickle("data")

In [ ]:
# pos_2d = [(x % grid_size, math.floor(x/grid_size), ) for x in range(grid_size*grid_size)]
# pos_mapping = dict(zip(range(grid_size*grid_size), pos_2d))
# pos_mapping
# nx.draw(G, pos_mapping)

In [ ]:
# create network graph and initialize nodes based on grid size 
G = nx.DiGraph(); 
for ride in range(data.shape[0]):
    src = data['pickup_zip'][ride]
    dest = data['dropoff_zip'][ride]
    G.add_edge(src, dest)
    


In [ ]:
# G.in_degree[]

In [ ]:
in_degree = np.zeros(100)
out_degree = np.zeros(100)

for k in G.nodes:
    in_num = G.in_degree[k]
    out_num = G.out_degree[k]
    in_degree[in_num] += 1
    out_degree[out_num] += 1
print(np.sum(in_degree))
print(np.sum(out_degree))
out_degree

In [ ]:
in_degree